# 0. Install Dependencies

In [ ]:
# !apt-get install --y ffmpeg freeglut3-dev xvfb  # For visualization

In [1]:
!pip install tensorflow-gpu
!pip install gym
!pip install keras
!pip install keras-rl2
!pip3 install gym_unity
# !pip install --force-reinstall git+git://github.com/openai/baselines@tf
!pip install stable-baselines3[extra]

     |████████████████████████████████| 52 kB 27 kB/s  eta 0:00:011


# 1. Test Random Environment with OpenAI Gym

In [1]:
import gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common.monitor import Monitor

In [2]:
env = Monitor(gym.make('CartPole-v1'))

model = PPO(MlpPolicy, env, verbose=0)

In [3]:
def evaluate(model, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            env.render()
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))
    
    env.close()
    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

In [5]:
# Random Agent, before training
mean_reward_before_train = evaluate(model, num_episodes=10)

Mean reward: 23.5 Num episodes: 10


In [6]:
from stable_baselines3.common.evaluation import evaluate_policy
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:21.29 +/- 8.90


In [7]:
# Train the agent for 10000 steps
model.learn(total_timesteps=10000)

In [8]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:303.79 +/- 116.83


In [9]:
# # Set up fake display; otherwise rendering will fail
# import os
# os.system("Xvfb :1 -screen 0 1024x768x24 &")
# os.environ['DISPLAY'] = ':0'

In [10]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [11]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param videos/o_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id)])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                                  record_video_trigger=lambda step: step == 0, video_length=video_length,
                                  name_prefix=prefix)

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

In [12]:
record_video('CartPole-v1', model, video_length=500, prefix='ppo2-cartpole')

In [13]:
show_videos('videos', prefix='ppo2')

In [12]:
try:
    from mpi4py import MPI
except ImportError:
    MPI = None

def make_unity_env(env_directory, num_env, visual, start_index=0):
    """
    Create a wrapped, monitored Unity environment.
    """
    def make_env(rank, use_visual=True): # pylint: disable=C0111
        def _thunk():
            unity_env = UnityEnvironment(env_directory, base_port=5000 + rank)
            env = UnityToGymWrapper(unity_env, uint8_visual=True)
            env = Monitor(env)
            return env
        return _thunk
    if visual:
        return SubprocVecEnv([make_env(i + start_index) for i in range(num_env)])
    else:
        rank = MPI.COMM_WORLD.Get_rank() if MPI else 0
        return DummyVecEnv([make_env(rank, use_visual=False)])

def main():
    env = make_unity_env("run27_linux", 4, False)
    ppo2.learn(
        network="mlp",
        env=env,
        total_timesteps=100000,
        lr=1e-3,
    )

if __name__ == '__main__':
    main()

NameError: name 'DummyVecEnv' is not defined

In [ ]:
env = gym.make('CartPole-v0')
env = UnityToGymWrapper(unity_env, uint8_visual, flatten_branched, allow_multiple_obs)

states = env.observation_space.shape[0]
actions = env.action_space.n

In [6]:
actions

2

In [7]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:19.0
Episode:2 Score:15.0
Episode:3 Score:12.0
Episode:4 Score:11.0
Episode:5 Score:67.0
Episode:6 Score:35.0
Episode:7 Score:29.0
Episode:8 Score:13.0
Episode:9 Score:15.0
Episode:10 Score:17.0


# 2. Create a Deep Learning Model with Keras

In [8]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [9]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [10]:
model = build_model(states, actions)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [12]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [13]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [21]:
del model
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 112s 11ms/step - reward: 1.0000
105 episodes - episode_reward: 93.733 [9.000, 200.000] - loss: 2.209 - mae: 18.578 - mean_q: 37.607

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 110s 11ms/step - reward: 1.0000
52 episodes - episode_reward: 194.827 [161.000, 200.000] - loss: 5.109 - mae: 38.310 - mean_q: 77.438

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 106s 11ms/step - reward: 1.0000
50 episodes - episode_reward: 196.800 [173.000, 200.000] - loss: 4.735 - mae: 41.417 - mean_q: 83.492

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 114s 11ms/step - reward: 1.0000
51 episodes - episode_reward: 199.745 [187.000, 200.000] - loss: 6.391 - mae: 40.217 - mean_q: 80.806

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 114s 11ms/step - reward: 1.

In [23]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
200.0


# 4. Reloading Agent from Memory

In [25]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [26]:
del model
del dqn
del env

In [27]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [28]:
dqn.load_weights('dqn_weights.h5f')

In [29]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
